In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
import time
import random
import pandas as pd
import re

#### 로그인 코드 

In [6]:
# 크롬 오픈
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(5)

# 틱톡 오픈
url = 'https://www.tiktok.com/ko-KR'
driver.get(url)
driver.implicitly_wait(5)

# 파서 지정
page = driver.page_source # page_source : selenium으로 불러온 페이지의 html코드 저장
# parser = BeautifulSoup(page,'html.parser')
# parser

# 로그인 버튼 클릭
driver.find_element(by='xpath', value='//*[@id="header-login-button"]/div').click()

# 전화/이메일/아이디 사용 버튼 클릭
driver.find_element(by='xpath', value='//*[@id="loginContainer"]/div/div/div/div[2]/div[2]').click()

# 이메일 또는 아이디로 로그인 클릭
driver.find_element(by='xpath', value='//*[@id="loginContainer"]/div/form/div[1]/a').click()

# 이메일 입력
input_email = driver.find_element(by='xpath', value='//*[@id="loginContainer"]/div[2]/form/div[1]/input')
input_email.send_keys('아이디 입력')

# 비밀번호 입력
input_password = driver.find_element(by='xpath', value='//*[@id="loginContainer"]/div[2]/form/div[2]/div/input')
input_password.send_keys('비밀번호 입력')

# 로그인버튼 클릭
driver.find_element(by='xpath', value='//*[@id="loginContainer"]/div[2]/form/button').click() #추후 돌아가는지 확인


#### 100개 : 좋아요, 댓글수, 저장수, 이름, 날짜, 설명, 해시태그, 조회수

In [24]:
# 검색 아이콘 클릭
wait = WebDriverWait(driver, 10)  # 최대 10초 기다림
search_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="app"]/div[2]/div[1]/div/div[2]/div[2]/button/div/div[2]'))).click()

# 검색어 입력
search = driver.find_element(By.XPATH, '//*[@id="app"]/div[2]/div[1]/div/div[5]/div[1]/div[2]/form/input')
search.send_keys('k-beauty')

# 키보드 엔터
search.send_keys(Keys.ENTER)

# 동영상 탭 클릭
driver.find_element(by='xpath', value='//*[@id="tabs-0-tab-search_video"]').click()

# 사람인거 인증하는 창 나옴 -> 대기 10초 걸어줌 / 최대한 빨리 수동으로 그림 맞추기 
driver.implicitly_wait(30)

# 페이지 로딩 대기 (동적으로 로드될 수 있음)
random
time.sleep()

# # 조회수 추출을 위해 스크롤 내리기
# for i in range(20):
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     time.sleep(2)
#     driver.find_element(By.TAG_NAME,'body').send_keys(Keys.ARROW_UP) # 위로 살짝 올렸다
#     driver.find_element(By.TAG_NAME,'body').send_keys(Keys.ARROW_DOWN) # 아래로 내려야 다음 게시물들이 로딩된다...ㅠㅠ
#     driver.implicitly_wait(5)

# 잠시 멈춤 
driver.implicitly_wait(10)

# # 조회수 100개만 추출
# views = driver.find_elements(By.CSS_SELECTOR, 'strong[data-e2e="video-views"]')
# view_list = []
# for view in views:
#     view_text = view.text
#     view_list.append(view_text)
# view_list = view_list[:100]

# 스크롤 맨 위로 올리기
driver.execute_script('window.scrollTo(0,0)')

time.sleep(2)

# 첫번 째 동영상 클릭
driver.find_element(By.XPATH, '//*[@id="tabs-0-panel-search_video"]/div/div/div[1]/div[1]').click()

# for문을 이용하여 n개의 게시물에서 좋아요수를 추출하여 리스트에 저장하기
like_list = []
comment_list = []
save_list = []
name_list = []
date_list = []
infos_tags_list =[]
hash_tag_list = []

n = 100

for i in range(n):

    try:
    # '자세히' 버튼이 존재하면 클릭
        expand_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'css-1r94cis-ButtonExpand'))
        )
        expand_button.click()
        print(i,'번:',"자세히 버튼 클릭 성공")
    except Exception as e:
        # 버튼이 없거나 클릭할 수 없을 경우 처리
        print(i,'번:',"자세히 버튼이 없거나 클릭할 수 없습니다.")

    # 영상설명 추출
    try:
        infos_tags = driver.find_element(By.CSS_SELECTOR, value = 'h1[data-e2e="browse-video-desc"]')
        infos_tags_text = infos_tags.text
        infos_tags_list.append(infos_tags_text)
    except NoSuchElementException:
        infos_tags_list.append('')   

    # # 해시태그 추출 
    # hash_tags = driver.find_elements(By.CSS_SELECTOR, value = 'a.css-sbcvet-StyledCommonLink.ejg0rhn6.link-a11y-focus strong')
    # hash_tag_text_list = []

    # for hash_tag in hash_tags: # 추출된 요소의 텍스트 출력
    #     hash_tag_text = hash_tag.text.strip()

    #     if hash_tag_text:
    #         hash_tag_text_list.append(hash_tag_text)
    #     else:
    #         continue
    # hash_tag_list.append(hash_tag_text_list)

    # 잠시 멈춤
    driver.implicitly_wait(5)

    #업로드 날짜 추출
    try:
        date = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.css-gg0x0w-SpanOtherInfos.evv7pft3'))
            )
        
        # 잠시 멈춤
        driver.implicitly_wait(5)

        date_text = date.text
        date_text = date_text.split('·')[1].strip()
        date_list.append(date_text)

    except StaleElementReferenceException:
        print("StaleElementReferenceException 발생: 요소가 더 이상 유효하지 않습니다.")
        date = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.css-gg0x0w-SpanOtherInfos.evv7pft3'))
            )
        
        # 잠시 멈춤
        driver.implicitly_wait(5)

        date_text = date.text
        date_text = date_text.split('·')[1].strip()
        date_list.append(date_text)
    
    except TimeoutException:
        print("업로드 날짜를 찾을 수 없습니다.")

    # 틱톡커 이름 추출 
    name = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'span[data-e2e="browse-username"]'))
    )
    name_text = name.text
    name_list.append(name_text)

    # 좋아요수 추출
    like = WebDriverWait(driver,10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'strong[data-e2e="browse-like-count"]'))
    )
    like_text = like[0].text #각 게시물에서 첫번째 요소만 처리/게시물 댓글의 좋아요까지 같은 셀렉터 이름을 사용함
    like_list.append(like_text)

    # 댓글수 추출
    comment = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'strong[data-e2e="browse-comment-count"]'))
    )
    comment_text = comment.text
    comment_list.append(comment_text)

    # 저장수 추출
    save = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'strong[data-e2e="undefined-count"]'))
    )
    save_text = save.text
    save_list.append(save_text)

    # 잠시 멈춤
    driver.implicitly_wait(3)

    #if i < n:
    driver.find_element(By.TAG_NAME,'body').send_keys(Keys.ARROW_DOWN)
    #    time.sleep(random.randint(2,5))

#print('조회수 리스트:',view_list)
print('좋아요수 리스트:',like_list)
print('댓글수 리스트:',comment_list)
print('저장수 리스트:',save_list)
print('틱톡커 이름 리스트:',name_list)
print('업로드 날짜 리스트:',date_list)
print('영상 설명,해시태그 리스트:',infos_tags_list)
#print('해시태그 리스트:',hash_tag_list)

0 번: 자세히 버튼 클릭 성공
1 번: 자세히 버튼 클릭 성공
2 번: 자세히 버튼 클릭 성공
3 번: 자세히 버튼 클릭 성공
4 번: 자세히 버튼 클릭 성공
5 번: 자세히 버튼 클릭 성공
6 번: 자세히 버튼 클릭 성공
7 번: 자세히 버튼 클릭 성공
8 번: 자세히 버튼 클릭 성공
9 번: 자세히 버튼 클릭 성공
10 번: 자세히 버튼 클릭 성공
11 번: 자세히 버튼이 없거나 클릭할 수 없습니다.
12 번: 자세히 버튼 클릭 성공
13 번: 자세히 버튼 클릭 성공
14 번: 자세히 버튼 클릭 성공
15 번: 자세히 버튼 클릭 성공
16 번: 자세히 버튼 클릭 성공
17 번: 자세히 버튼이 없거나 클릭할 수 없습니다.
18 번: 자세히 버튼이 없거나 클릭할 수 없습니다.
19 번: 자세히 버튼이 없거나 클릭할 수 없습니다.
20 번: 자세히 버튼 클릭 성공
21 번: 자세히 버튼 클릭 성공
22 번: 자세히 버튼 클릭 성공
23 번: 자세히 버튼 클릭 성공
24 번: 자세히 버튼 클릭 성공
25 번: 자세히 버튼 클릭 성공
26 번: 자세히 버튼 클릭 성공
27 번: 자세히 버튼 클릭 성공
28 번: 자세히 버튼 클릭 성공
29 번: 자세히 버튼 클릭 성공
30 번: 자세히 버튼 클릭 성공
31 번: 자세히 버튼이 없거나 클릭할 수 없습니다.
32 번: 자세히 버튼 클릭 성공
33 번: 자세히 버튼 클릭 성공
34 번: 자세히 버튼 클릭 성공
35 번: 자세히 버튼 클릭 성공
36 번: 자세히 버튼 클릭 성공
37 번: 자세히 버튼 클릭 성공
38 번: 자세히 버튼 클릭 성공
39 번: 자세히 버튼 클릭 성공
40 번: 자세히 버튼이 없거나 클릭할 수 없습니다.
41 번: 자세히 버튼이 없거나 클릭할 수 없습니다.
42 번: 자세히 버튼 클릭 성공
43 번: 자세히 버튼 클릭 성공
44 번: 자세히 버튼이 없거나 클릭할 수 없습니다.
45 번: 자세히 버튼 클릭 성공
46 번: 자세히 버튼 클릭 성공
47 번: 자세히 버튼 클릭 성공
48 번: 자세히 

In [25]:
#print('조회수 리스트:',len(view_list))
print('좋아요수 리스트:',len(like_list))
print('댓글수 리스트:',len(comment_list))
print('저장수 리스트:',len(save_list))
print('틱톡커 이름 리스트:',len(name_list))
print('업로드 날짜 리스트:',len(date_list))
print('영상 설명,해시태그 리스트:',len(infos_tags_list))
#print('해시태그 리스트:',len(hash_tag_list))

조회수 리스트: 100
좋아요수 리스트: 100
댓글수 리스트: 100
저장수 리스트: 100
틱톡커 이름 리스트: 100
업로드 날짜 리스트: 100
영상 설명,해시태그 리스트: 100
해시태그 리스트: 100


#### 데이터 프레임으로 만들기

In [28]:
# 데이터 프레임으로 만들기

df = pd.DataFrame({#'view':view_list,
'like':like_list,
'comment':comment_list,
'save':save_list,
'titoker_name':name_list,
'date':date_list,
'info':infos_tags_list,
#'hash_tag':hash_tag_list
})

df

,view,like,comment,save,titoker_name,date,info,hash_tag
0,180K,180K,539,47.6K,luv2jna,2024-4-3,Replying to @Farah❤️ the one thing i can yap a...,"[@Farah❤️, @kiokiicanada, @Farah❤️, #makeup, #..."
1,558.6K,558.6K,4674,65.8K,amuse_official,2024-10-16,🎀AMUSE & AMUSE GIRL🎀 let's SHAKE and TOK! TOK!...,"[@Farah❤️, @kiokiicanada, #amusemakeup, #amuse..."
2,449.3K,449.3K,2842,69.5K,ashlynbeauty_,2024-5-9,@kiokiicanada is every girls dream🤍 they have ...,"[@Farah❤️, @kiokiicanada, @kiokiicanada, #kore..."
3,332.3K,332.3K,4289,74.3K,bwnniewony,2024-8-23,My secret to soft skin ! 🍓🎀 - You can find thi...,"[@Farah❤️, @kiokiicanada, #skincare, #kbeauty,..."
4,146.6K,146.6K,1628,23.1K,mydelicate,2024-4-11,mini pink makeup tour because i am reorganisin...,"[@Farah❤️, @kiokiicanada, #makeup, #koreanbeau..."
...,...,...,...,...,...,...,...,...
95,646,646,64,103,rosiehashs,1d ago,spend a day with me grwm + shopping💗🎀 mini vlo...,"[@Farah❤️, @kiokiicanada, #you, @Anua Store US..."
96,16.8K,16.8K,127,2771,4chriisty,2024-5-7,use my yesstyle rewards code CRISPY5 to save m...,"[@Farah❤️, @kiokiicanada, #fyp, #fypage, #govi..."
97,180,180,35,13,growsattraderjoes,1-11,Anua new packaging for their peach line is … d...,"[@Farah❤️, @kiokiicanada, #wonyoungism, #anua,..."
98,33.2K,33.2K,662,6909,4ngeljeon,2024-9-17,˚｡⋆୨୧˚ — all the girls are girling girling 🧸.ᐟ...,"[@Farah❤️, @kiokiicanada, #aesthetic, #pink, #..."


In [29]:
df.to_csv('tiktok_post_k_beauty_0121',index=False, encoding="utf-8-sig")

In [1]:
#다음 전처리 해주기 : 처음n개의 해시태그 빼주기
#tag_df['hash_tag'][0]
#del tag_df['hash_tag'][0][0] 
#print(tag_df['hash_tag'][0])